### IDs:
Insert yours IDs to the cell below

ID #1:310116165

ID #2:308564293


## Read the following instructions carefully:

1. This jupyter notebook contains all the step by step instructions needed for this exercise.
1. You are free to add cells.
1. Write your functions and your answers in this jupyter notebook only.
1. Answers to theoretical questions should be written in **markdown cells (with $\LaTeX$ support)**.
1. Submit this jupyter notebook only using your ID as a filename. Not to use ZIP or RAR. For example, your Moodle submission file name should look like this (two id numbers): `123456789_987654321.ipynb`.

### Question 1 - Data exploration and visialization - practical

Load Boston dataset from sklearn.
Explore the data. follow th instructions below and make sure to support your answers with proper outputs and plots.
When plotting, pay close attention to the range of the axis, and include axis labels and a title for the figure.

1. Describe the dataset. How many samples does it contain? How many features? What is the data type for each variable?
2. Produce a histogram and a boxplot of the nitric oxides concentration. describe the distribution.
3. Produce a correlation matrix of all the features. Are there any correlated features? Can you identify one feature with unusual behaviour?
4. Select the 2 pairs of features with the highest correlation (positive or negative) and plot 2 scatter plots with marginal histograms (JointPlot). 
5. Produce a cumulative histogram of the age variable and add two horizontal lines on the first and third quartile (on the cumulative count)
6. Identify and report 2 “interesting” trends in the data. No need to provide statistical confidence at this point. 

In [1]:
# <!-- <p style="color:red;"> -->
%pylab inline 
import pandas as pd

df=pd.read_...
# answer to 1 
df.info()
df.describe()
# answer to 2
df.nitric_oxide_concentration.hist(kind="hist")
df.nitric_oxide_concentration.plot(kind="box")

# answer to 3
corr_mat = df.corr()

# answer to 4
most_extreme_pair = corr_mat.stack().apply(np.abs).idxmax()
# some code that does joint plot.. 
# answer to 5 
age_cumsum = df.age.sort_values().cumsum()
age_cumsum.hist()
first_quartile = np.quantile(age_cumsum,0.25)
third_quartile = np.quantile(age_cumsum,0.75)
plt.plot([first_quartile]*2,[0,1],'r')
plt.plot([third_quartile]*2,[0,1],'r')
# answer to 6
# find trends by correlation, value counts. maybe we can copy some known trends from the web

SyntaxError: invalid syntax (1165197209.py, line 5)

### Question 2 - Independence and conditional independence

#### 2.A
Let $\ X, Y \ $ and $Z$  be discrete random variables with $\ n, m \ $ and $k=4$ possible outcomes respectivley.

How many parameters define the joint distribution of $\ X, Y \ $ and $Z$?

<p style="color:red;">
    Given no additional information, we need to know all possible outputs to to define the distribution and also all their probabilities (-1 because we have a constraint on the sum of probabilities). 
    which means we need (2n-1) + (2m-1) + (2k-1) parameters.
    The joint distribution is defined by the same amount of parameters. 

#### 2.B
For the same random variables from the previous section, how many parameters define the joint distribution of $\ X, Y \ $ and $Z$ if we now know that they are independent?

<p style="color:red;">
    Same answer. Knowing they are independent does not provide with additional constraints

#### 2.C
For the same random variables from the previous section, how many parameters define the joint distribution of $\ X, Y \ $ and $Z$ if we now know that $X$ and $Y$ are conditionaly independent given $Z$?

#### 2.D
Give an example for a joint distribution of $\ U, V \ $ and $W$ where $U$ and $V$ are NOT conditionally independent given $W$, but $U$ and $V$ are (unconditionally) independent. Also: $U$ and $V$ are standard normally distributed ($N(0, 1)$) and $W$ is a coin (2 possible values).

### Question 3 - Gaussian mixtures – parameter estimation and generation 

Consider the data provided in GMD.csv
Assume that the data comes from a Gaussian mixture distribution (GMD) with $k=3$. Furthermore, assume that $\mu_{1}=4, \mu_{2}=9, \sigma_{1}=\sigma_{2}=0.5, \sigma_3=1.5$ and $w_2=0.25$.

Read the data and answer the following questions.

#### 3.A
Provide an estimate for the other parameters of the distribution in two different ways.

<p style="color:red;">
    - 1) chi2 minimization (we have 3 unknown parameters) 
    - 2) depending on the data we might be able to see the peaks and get the weights fomr them (1 and 2 are well seperated (at least one of them is) so it should be possible)
    - 3) not sure what other ways there are... 

#### 3.B
Plot a graph of the pdf of the distribution you inferred. Select adequate limits for the axes for this plot and explain your decision.

<p style="color:red;">
    we just need to plot the function we retrieve.. 

#### 3.C
Now assume that the data comes from a Gaussian mixture distribution (GMD) with $k=4$.

The given data and parameters stay the same.

Can you estimate the unknown parameters in the two ways described in section A? Explain.

<p style="color:red;">
- 1) possible by adding 2 degrees of freedom and try to minimize chi2 farther?
- 2) only by guessing we in the data we see another peak. Probably a bad idea unless it will be very clear the peak is there. 

#### 3.D
Describe two ways for generating data for a GMM random variable with:
* centers at  $\mu_1=3, \mu_2=7, \mu_3=10$
* $\sigma_1=\sigma_2=\sigma_3=1$
* $w_1=w_2=w_3=0.33$

<p style="color:red;">
- 1) Monte Carlo (MC) - randomly choose x and pick it if some random value p bigger than the pdf?
- 2) Create 3 different distributions (using the same MC method) and then merge them together? In this case it is a simple combination because the weights are identical
- 3) Something other than MC

#### 3.E
Use one of the above approaches to generate 1000 points and plot a histogram of the result (decide on bins, axes etc)


#### 3.F
Use the other one to generate 1000 more points and draw two comparative histograms.

### Question 4 - Normally distributed salaries

The annual salaries of employees in a large Randomistan company are approximateley normally distributed with a mean of 75,000 RCU and a standard deviation of 37,500 RCU.

#### 4.A
What percent of people earn less than 50,000 RCU?

In [ ]:
stats.norm(loc=75000,?=37500).cdf(50_000)

#### 4.B
What percent of people earn between 45,000 RCU and 65,000 RCU?

In [ ]:
(stats.norm(loc=75000,?=37500).cdf(65_000) - stats.norm(loc=75000,?=37500).cdf(45_000))*100

#### 4.C
What percent of people earn more than 85,000 RCU?

In [ ]:
1-stats.norm(loc=75000,?=37500).cdf(85_000)

#### 4.D
The company has 1000 employees. How many employees in the company do you expect to earn more than 140,000 RCU?

In [ ]:
(1-stats.norm(loc=75000,?=37500).cdf(140_000)) * 1000

### Question 5 - Coupon collector

Let $T_{N}$ denote the waiting time for full single coupon collection with N different equiprobable coupon types

#### 5.A
Write code to compute the exact value of $E(T_{N})$

#### 5.B
Write code to compute the exact value of $V(T_{N})$

#### 5.C
Write code to exactly compute $P(T_{30}>60)$

#### 5.D
Let $S_N$ denote the waiting time until $N/2$ different coupons are collected.

Write code to exactly compute $P(S_{30}>60)$